<a href="https://colab.research.google.com/github/ashworks1706/llm-from-scratch/blob/main/projects/01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 96.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# 1. Load Model and Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-bnb-4bit", # Or "unsloth/Qwen2.5-1.5B-bnb-4bit"
    max_seq_length = 2048,
    load_in_4bit = True,
)

==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank: 8, 16, 32 are common. Higher = more capacity.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Optimized for 0
    bias = "none",    # Optimized for "none"
    use_gradient_checkpointing = "unsloth", # Saves massive VRAM
    random_state = 3407,
)

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

In [6]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [7]:
dataset = load_dataset("Magpie-Align/Magpie-Pro-300K-Filtered", split = "train")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00003.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

data/train-00001-of-00003.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

data/train-00002-of-00003.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300000 [00:00<?, ? examples/s]

In [8]:
dataset = standardize_sharegpt(dataset)


Unsloth: Standardizing formats (num_proc=6):   0%|          | 0/300000 [00:00<?, ? examples/s]

In [9]:
dataset

Dataset({
    features: ['conversations', 'uuid'],
    num_rows: 300000
})

In [10]:
print(dataset[0])

{'conversations': [{'content': 'Can you explain the concept of equitable distribution in family law?', 'role': 'user'}, {'content': 'In family law, equitable distribution refers to the process of dividing marital assets and debts fairly and reasonably between spouses in the event of a divorce or separation. The goal of equitable distribution is to allocate the marital estate in a way that is just and equitable, considering the unique circumstances of each case.\n\nHere are some key principles and concepts related to equitable distribution:\n\n1. **Marital Property**: Marital property includes all assets and debts acquired during the marriage, regardless of whose name they are in. This can include real estate, bank accounts, investments, retirement accounts, vehicles, and personal property.\n2. **Separate Property**: Separate property, on the other hand, refers to assets and debts acquired before the marriage, inherited, or gifted to one spouse individually. These assets are typically n

In [11]:
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/300000 [00:00<?, ? examples/s]

In [12]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Small test run first!
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        output_dir = "outputs",
    ),
)



Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/300000 [00:00<?, ? examples/s]

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 300,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: Enter your choice:wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,1.061300
2,0.913800
3,0.981100
4,0.924700
5,1.022400
6,1.001800
7,0.962800
8,0.949800
9,0.816100
10,0.883300


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▆▅▆█▇▆▇▅▄▃▂▁▂▂▃▃▂▂▁▁▁▂▁▂▁▂▂▁▂▁▂▁▂▁▁▂▁▁▂▂
train/learning_rate,▁▂▄▅▇████▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁
train/loss,█▆▆▇▆▅▅▂▁▃▄▂▃▃▄▄▄▂▂▃▂▂▄▂▃▂▃▁▂▂▂▂▁▃▂▄▂▂▂▃
total_flos,1.5626143017504768e+16
train/epoch,0.0016
train/global_step,60
train/grad_norm,0.14539
train/learning_rate,0.0
train/loss,0.7875


TrainOutput(global_step=60, training_loss=0.7745601028203964, metrics={'train_runtime': 1033.0637, 'train_samples_per_second': 0.465, 'train_steps_per_second': 0.058, 'total_flos': 1.5626143017504768e+16, 'train_loss': 0.7745601028203964, 'epoch': 0.0016})

In [14]:
from transformers import TextStreamer

# 1. Enable native 2x faster inference
FastLanguageModel.for_inference(model) 

# 2. Setup a streamer to see the text as it's being generated (one word at a time)
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

In [15]:
# 3. Create your test message
messages = [
    {"role": "user", "content": "Explain the concept of knowledge distillation to a 10-year-old."},
]

# 4. Apply the template and move to GPU
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # This tells the model it's its turn to talk
    return_tensors = "pt",
).to("cuda")

# 5. Generate!
_ = model.generate(
    input_ids = inputs, 
    streamer = text_streamer, 
    max_new_tokens = 512,
    use_cache = True,    # Essential for fast inference
    temperature = 0.7,   # Control creativity
    min_p = 0.1          # Recommended for Qwen 2.5
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Knowledge distillation is like teaching a simple recipe to a friend, so they can make a delicious cake without having to learn all the complicated steps of baking from scratch.

Imagine you have a big, fancy cake recipe that's really hard to follow. It has lots of ingredients and steps, and it takes a long time to make. Now, you want to teach your friend how to make a cake, but you don't want to overwhelm them with all the complicated steps.

So, you decide to simplify the recipe. You pick out the most important ingredients and steps, and you explain them in a way that's easy to understand. You might even use pictures or videos to help your friend see what to do.

This is what knowledge distillation is like. It's when we take a big, complicated model (like a fancy cake recipe) and we simplify it, so it's easier to understand and use. We do this by picking out the most important parts of the model and teaching them to a smaller, simpler model (like a friend learning to make a cake).

Th

In [ ]:
model.save_pretrained("./qwen_student_model")
tokenizer.save_pretrained("./qwen_student_model")
model.save_pretrained_merged("model_fused", tokenizer, save_method = "merged_16bit")

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

Unsloth: Preparing safetensor model files:  75%|███████▌  | 3/4 [04:13<01:20, 80.61s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [04:26<00:00, 66.74s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import os
os.system("zip -r qwen_model.zip ./qwen_student_model")

bin			    kaggle		      opt		 sys
boot			    lib			      proc		 tmp
content			    lib32		      python-apt	 tools
cuda-keyring_1.1-1_all.deb  lib64		      python-apt.tar.xz  usr
datalab			    libx32		      root		 var
dev			    media		      run
etc			    mnt			      sbin
home			    NGC-DL-CONTAINER-LICENSE  srv


In [ ]:
!ls

helo world
